Lớp: NT204.O21.ATCL.2- Nhóm 3

Thành viên:

  Huỳnh Minh Tân Tiến - 21521520
  
  Lê Hoàng Oanh - 21521253

In [ ]:
# Load Drive Helper
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model

In [ ]:
# Đường dẫn đến thư mục dataset
DATASET_PATH = '/content/drive/MyDrive/NT204/dataset/'
TRAINSET_PATH = DATASET_PATH + 'train_set.csv'
TESTSET_PATH = DATASET_PATH + 'test_set.csv'

# Tạo thư mục để tải dataset nếu chưa tồn tại
if not os.path.exists(DATASET_PATH):
  os.makedirs(DATASET_PATH)

'''
Tải tập train (train_set.csv) và tập test (test_set.csv) vào thư mục /content/drive/My Drive/NT204/dataset/ trên drive.
Lưu ý: Đặt đúng tên các file.
- Tải các tập dữ liệu tại: https://drive.google.com/drive/folders/1nYyDqbmichotvI9eSOrKo-yRwtLUy_iM?usp=sharing
'''


'\nTải tập train (train_set.csv) và tập test (test_set.csv) vào thư mục /content/drive/My Drive/NT204/dataset/ trên drive.\nLưu ý: Đặt đúng tên các file.\n- Tải các tập dữ liệu tại: https://drive.google.com/drive/folders/1nYyDqbmichotvI9eSOrKo-yRwtLUy_iM?usp=sharing\n'

In [ ]:
# Tập train
data_train = pd.read_csv(TRAINSET_PATH)
data_train

,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,2.434185e-06,0.888060,0.400000,1.251901e-09,0.000074,0.000000,0.996078,0.000000,0.000000,0.000000,...,0.0,0.000000,0.484849,0.484849,0.212121,0.212121,0.212121,0.000000,0.484849,Normal
1,1.539919e-07,0.888060,0.333333,2.276184e-10,0.000018,0.000000,0.235294,0.000000,0.000000,0.000000,...,0.0,0.000000,0.560606,0.560606,0.560606,0.560606,0.560606,0.288136,0.560606,Normal
2,1.539919e-07,0.888060,0.333333,3.414275e-10,0.000008,0.000000,0.996078,0.000000,0.000000,0.000000,...,0.0,0.000000,0.348485,0.348485,0.227273,0.227273,0.227273,0.254237,0.348485,Generic
3,1.539919e-07,0.888060,0.133333,1.190444e-07,0.000009,0.000011,0.121569,0.114173,0.000000,0.000000,...,0.2,0.111111,0.045455,0.060606,0.030303,0.121212,0.000000,0.000000,0.000000,Normal
4,1.539919e-07,0.888060,0.333333,5.690459e-10,0.000018,0.000000,0.235294,0.000000,0.000000,0.000000,...,0.0,0.000000,0.469697,0.469697,0.348485,0.348485,0.348485,0.389831,0.469697,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778027,1.539919e-07,0.888060,0.333333,1.138092e-09,0.000008,0.000000,0.996078,0.000000,0.000000,0.000000,...,0.0,0.000000,0.651515,0.651515,0.348485,0.348485,0.348485,0.389831,0.651515,Generic
1778028,1.539919e-07,0.888060,0.333333,9.104734e-10,0.000018,0.000000,0.235294,0.000000,0.000000,0.000000,...,0.0,0.000000,0.348485,0.348485,0.348485,0.348485,0.348485,0.389831,0.348485,Normal
1778029,3.657512e-04,0.843284,0.266667,9.668203e-06,0.000339,0.005373,0.121569,0.114173,0.001316,0.006537,...,0.0,0.000000,0.136364,0.090909,0.075758,0.121212,0.000000,0.000000,0.000000,Normal
1778030,3.829222e-04,0.843284,0.333333,2.048451e-06,0.000170,0.001333,0.121569,0.114173,0.001316,0.002361,...,0.2,0.111111,0.045455,0.060606,0.015152,0.030303,0.000000,0.000000,0.000000,Normal


In [ ]:
# Load dữ liệu để kiểm tra mô hình
data_test = pd.read_csv(TESTSET_PATH)
data_test

,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,1.539919e-07,0.888060,0.333333,7.966642e-10,0.000018,0.000000,0.235294,0.000000,0.000000,0.000000,...,0.0,0.000000,0.787879,0.787879,0.621212,0.621212,0.621212,0.305085,0.787879,Normal
1,3.873917e-05,0.843284,0.266667,6.163905e-07,0.000144,0.000165,0.121569,0.114173,0.001128,0.001271,...,0.0,0.000000,0.075758,0.106061,0.015152,0.015152,0.000000,0.000000,0.045455,Normal
2,1.539919e-07,0.888060,0.333333,7.966642e-10,0.000018,0.000000,0.235294,0.000000,0.000000,0.000000,...,0.0,0.000000,0.166667,0.166667,0.166667,0.166667,0.166667,0.186441,0.166667,Normal
3,8.712603e-06,0.850746,0.333333,2.548848e-05,0.000622,0.000022,0.121569,0.114173,0.000752,0.000182,...,0.2,0.111111,0.015152,0.015152,0.015152,0.030303,0.000000,0.000000,0.015152,Normal
4,5.992697e-07,0.850746,0.333333,3.831909e-05,0.000544,0.001026,0.121569,0.114173,0.005640,0.005811,...,0.2,0.111111,0.045455,0.030303,0.015152,0.000000,0.000000,0.000000,0.000000,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762010,1.539919e-07,0.888060,0.333333,2.276184e-10,0.000008,0.000000,0.996078,0.000000,0.000000,0.000000,...,0.0,0.000000,0.500000,0.500000,0.242424,0.242424,0.242424,0.271186,0.500000,Generic
762011,4.963549e-05,0.843284,0.266667,6.210544e-05,0.000733,0.035276,0.121569,0.114173,0.002820,0.033049,...,0.0,0.000000,0.151515,0.060606,0.000000,0.000000,0.000000,0.000000,0.015152,Normal
762012,2.374815e-07,0.843284,0.266667,6.856338e-04,0.001345,0.074221,0.121569,0.114173,0.000376,0.067187,...,0.0,0.000000,0.015152,0.000000,0.015152,0.015152,0.000000,0.000000,0.000000,Normal
762013,3.873917e-05,0.843284,0.333333,6.709051e-07,0.000134,0.000294,0.121569,0.114173,0.001128,0.001090,...,0.0,0.000000,0.060606,0.075758,0.015152,0.015152,0.015152,0.000000,0.015152,Normal


In [ ]:
# Đọc dữ liệu train và test từ các file CSV
data_train = pd.read_csv(TRAINSET_PATH)
data_test = pd.read_csv(TESTSET_PATH)

# Tiền xử lý dữ liệu
label_encoder = preprocessing.LabelEncoder()

# Tách features (x) và labels (y) từ dữ liệu train
x_train, y_train = data_train.iloc[:, :-1], data_train.iloc[:, [-1]]
y_train = y_train.apply(label_encoder.fit_transform)  # Mã hóa nhãn
x_train, y_train = x_train.to_numpy(), y_train.to_numpy()  # Chuyển đổi sang mảng numpy

# Tách features (x) và labels (y) từ dữ liệu test
x_test, y_test = data_test.iloc[:, :-1], data_test.iloc[:, [-1]]
y_test = y_test.apply(label_encoder.fit_transform)  # Mã hóa nhãn
x_test, y_test = x_test.to_numpy(), y_test.to_numpy()  # Chuyển đổi sang mảng numpy

# Chuẩn hóa dữ liệu
scaler = preprocessing.MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Lấy số thuộc tính và số lớp
NUM_FEATURES = x_train.shape[1]
NUM_CLASSES = np.unique(y_train).size

# One-hot encoding cho nhãn
y_train = tf.keras.utils.to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=NUM_CLASSES)

In [ ]:
print("Number of features: %s" %NUM_FEATURES)
print("Number of classes: %s" %NUM_CLASSES)

Number of features: 42
Number of classes: 11


In [ ]:
'''
Mô hình ResNet này bao gồm các thành phần chính như sau:

1. **Lớp đầu vào (Input layer):**
   - Định nghĩa kích thước của đầu vào là (NUM_FEATURES, 1), trong đó NUM_FEATURES là số lượng đặc trưng của dữ liệu.

2. **Lớp tích chập ban đầu (Initial Conv1D layer):**
   - Lớp này sử dụng 64 filters với kích thước kernel là 3 để trích xuất các đặc trưng cơ bản từ dữ liệu đầu vào.
   - Kết hợp với BatchNormalization để tăng tốc độ huấn luyện và ổn định mô hình.
   - Sử dụng MaxPooling1D để giảm chiều dài của đầu vào, giúp giảm kích thước và tính toán của mô hình.

3. **Các khối residual (Residual blocks):**
   - Mô hình bao gồm 6 khối residual, mỗi khối sử dụng 64 filters.
   - Mỗi khối residual bao gồm hai lớp tích chập 1D, các lớp BatchNormalization và ReLU activation.
   - Khối residual giúp mô hình học được các đặc trưng phức tạp hơn và giảm thiểu hiện tượng mất mát thông tin khi chiều sâu của mạng tăng lên (vanishing gradient).

4. **Lớp làm phẳng và fully-connected (Flatten and Dense layers):**
   - Đầu ra của khối residual cuối cùng được làm phẳng thành vector.
   - Lớp fully-connected với 256 neurons và ReLU activation giúp học các đặc trưng từ đầu ra của các khối residual.
   - Sử dụng Dropout với tỉ lệ 0.5 để giảm overfitting, bằng cách ngẫu nhiên bỏ qua một số neurons trong quá trình huấn luyện.

5. **Lớp đầu ra (Output layer):**
   - Lớp đầu ra sử dụng số lượng neurons bằng số lớp (NUM_CLASSES) và softmax activation để tạo ra xác suất dự đoán cho mỗi lớp.

Mô hình được compile với Adam optimizer và categorical crossentropy loss function. Adam optimizer tự động điều chỉnh tốc độ học, giúp quá trình huấn luyện nhanh chóng và hiệu quả hơn. Categorical crossentropy là hàm mất mát phổ biến cho bài toán phân loại nhiều lớp, giúp tối ưu hóa độ chính xác của mô hình.

'''

# Định nghĩa khối residual với BatchNormalization
def residual_block(x, filters, kernel_size=3):
    shortcut = x  # Lưu lại đầu vào để làm đường tắt (shortcut)

    # Lớp tích chập 1D đầu tiên trong khối residual
    x = Conv1D(filters, kernel_size=kernel_size, padding='same', activation=None)(x)
    x = BatchNormalization()(x)  # Lớp chuẩn hóa theo batch để ổn định và tăng tốc độ huấn luyện
    x = Activation('relu')(x)  # Lớp kích hoạt ReLU để thêm tính phi tuyến tính

    # Lớp tích chập 1D thứ hai trong khối residual
    x = Conv1D(filters, kernel_size=kernel_size, padding='same', activation=None)(x)
    x = BatchNormalization()(x)  # Lớp chuẩn hóa theo batch

    # Cộng kết quả với shortcut để tạo ra kết nối ngắn (skip connection)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)  # Lớp kích hoạt ReLU cuối cùng
    return x  # Trả về đầu ra của khối residual

# Xây dựng mô hình ResNet
input_layer = Input(shape=(NUM_FEATURES, 1))  # Lớp đầu vào với kích thước bằng số thuộc tính và 1 kênh (channel)

# Lớp tích chập 1D ban đầu với 64 filters, kích thước kernel là 3 và ReLU activation
x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)  # Lớp chuẩn hóa theo batch
x = MaxPooling1D(pool_size=2)(x)  # Lớp pooling 1D với kích thước pool là 2 để giảm chiều dài của đầu vào

# Thêm các khối residual
for _ in range(6):  # Tạo 6 khối residual
    x = residual_block(x, 64)  # Mỗi khối residual có 64 filters

# Làm phẳng đầu ra từ khối residual cuối cùng thành vector để đưa vào lớp fully-connected
x = Flatten()(x)
# Lớp fully-connected với 256 neurons và ReLU activation để học các đặc trưng phức tạp từ đầu ra của các khối residual
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Lớp dropout với tỉ lệ 0.5 để giảm overfitting bằng cách ngẫu nhiên bỏ qua một số neurons trong quá trình huấn luyện
# Lớp đầu ra với số neurons bằng số lớp và softmax activation để tạo ra xác suất dự đoán cho mỗi lớp
output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

# Tạo mô hình với đầu vào và đầu ra đã định nghĩa
model_resnet = Model(inputs=input_layer, outputs=output_layer)
# Compile mô hình với Adam optimizer và loss function là categorical crossentropy
# Chúng ta sử dụng Adam optimizer vì nó tự động điều chỉnh tốc độ học trong quá trình huấn luyện, giúp tối ưu hóa hiệu quả hơn
# Loss function categorical crossentropy phù hợp với bài toán phân loại nhiều lớp (multi-class classification)
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Hiển thị mô hình
model_resnet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 42, 1)]              0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 42, 64)               256       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 42, 64)               256       ['conv1d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 21, 64)               0         ['batch_normalization[0][0

In [ ]:
# Huấn luyện mô hình
model_resnet.fit(x=x_train, y=y_train, batch_size=1024, epochs=30, shuffle=True)

Epoch 1/30
1737/1737 [==============================] - 81s 37ms/step - loss: 0.0860 - accuracy: 0.9701
Epoch 2/30
1737/1737 [==============================] - 64s 37ms/step - loss: 0.0663 - accuracy: 0.9743
Epoch 3/30
1737/1737 [==============================] - 65s 37ms/step - loss: 0.0626 - accuracy: 0.9755
Epoch 4/30
1737/1737 [==============================] - 64s 37ms/step - loss: 0.0602 - accuracy: 0.9765
Epoch 5/30
1737/1737 [==============================] - 64s 37ms/step - loss: 0.0586 - accuracy: 0.9770
Epoch 6/30
1737/1737 [==============================] - 64s 37ms/step - loss: 0.0571 - accuracy: 0.9776
Epoch 7/30
1737/1737 [==============================] - 64s 37ms/step - loss: 0.0559 - accuracy: 0.9778
Epoch 8/30
1737/1737 [==============================] - 64s 37ms/step - loss: 0.0550 - accuracy: 0.9780
Epoch 9/30
1737/1737 [==============================] - 64s 37ms/step - loss: 0.0544 - accuracy: 0.9782
Epoch 10/30
1737/1737 [==============================] - 64s 37m

In [ ]:
# Đánh giá mô hình
preds = model_resnet.predict(x_test, batch_size=1024)
y_pred = np.argmax(preds, axis=1)
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
precision = precision_score(y_test.argmax(axis=1), y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test.argmax(axis=1), y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_test.argmax(axis=1), y_pred, average='weighted', zero_division=0)

print('ResNet - Accuracy: %s\nPrecision: %s\nRecall: %s\nF1-Score: %s' % (accuracy, precision, recall, f1))

745/745 [==============================] - 8s 10ms/step
ResNet - Accuracy: 0.9802576064775628
Precision: 0.980812515764382
Recall: 0.9802576064775628
F1-Score: 0.9778605870034145


In [ ]:
# Lưu mô hình
model_resnet.save('/content/drive/My Drive/NT204/model-resnet.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
